# How robo-advisors make investment decisions

The examples below demonstrate the basic concept of how robo-advisors make investment decisions:

- Basic portfolio optimization using Modern Portfolio Theory (covered in much greater detail last time)
- **Tax-loss harvesting logic** (THIS FILE)
- Risk profiling and asset allocation
- Portfolio rebalancing

We will use these examples to:

- Demonstrate how robo-advisors make investment decisions
- Show how risk profiles are translated into portfolio allocations
- Illustrate tax-loss harvesting logic
- Demonstrate portfolio rebalancing mechanics

Each example can be expanded. You may want to try:

- Add additional (or interactive) visualization using `matplotlib` or `plotly`
- Include more sophisticated optimization techniques (think `riskfolio` package)
- Add error handling and input validation (useful for more advanced and user-friendly applications)
- Include more complex tax rules
- Add additional real-time market data integration (more stocks, different time periods, alternative assets).

## Tax Loss Harvesting: An Interactive Example

Tax Loss Harvesting (TLH) is an investment strategy that can help reduce an investor's tax liability while maintaining their desired market exposure. This example demonstrates the key concepts and calculations involved in implementing a basic TLH strategy.

### What is Tax Loss Harvesting?

Tax loss harvesting is the practice of selling securities at a loss to offset capital gains tax liability. By realizing these losses, investors can:
1. Offset current capital gains
2. Reduce ordinary taxable income (up to $3,000 per year)
3. Bank excess losses for future tax years

### Key Components

#### 1. Wash Sale Rule
- The IRS (and many other tax authorities) prohibits claiming a loss on a security if you purchase the same or "substantially identical" security within 30 days before or after the sale
- This is known as the "wash sale rule"
- Our implementation checks the holding period to avoid wash sales
- In practice, robo-advisors often use similar but not identical securities (e.g., different ETFs tracking similar indices) to maintain market exposure

#### 2. Loss Threshold
- Not every loss should trigger harvesting
- Transaction costs and minimum trade sizes need to be considered
- Our implementation uses a loss threshold parameter to determine when harvesting is worthwhile
- Typical thresholds range from 1% to 15% depending on:
  * Transaction costs
  * Tax rates
  * Expected price recovery
  * Trading frequency preferences

#### 3. Tax Rate Considerations
- Different tax rates apply to different investors
- Short-term vs. long-term capital gains rates (check ATO website)
- State and local taxes may apply (generally, applicable in US/Canada where provintial and/or state taxes are separate from federal taxes)
- Our implementation allows adjustment of tax rates to explore different scenarios

## Implementation Details

### Portfolio Structure
Our example portfolio contains multiple tax lots with:
- Security identifier
- Quantity
- Purchase price
- Current price
- Purchase date

This mimics how actual portfolios track tax lots separately, even for the same security.

### Key Calculations

#### 1. Unrealized Gain/Loss
```python
unrealized_gain = (current_price - purchase_price) / purchase_price
```
- Expressed as a percentage
- Negative values represent harvestable losses
- Used to compare against threshold

#### 2. Dollar Loss
```python
dollar_loss = (current_price - purchase_price) * quantity
```
- Absolute dollar amount of the loss
- Used to calculate potential tax savings

#### 3. Tax Savings
```python
tax_savings = abs(dollar_loss) * tax_rate
```
- Potential tax reduction from harvesting the loss
- Assumes losses can be used (either against gains or income)

### Interactive Parameters

#### 1. Tax Rate Slider (10% - 50%)
- Represents combined federal and state tax rates
- Higher tax rates increase the value of tax loss harvesting
- Typical values:
  * 15-20% for long-term capital gains
  * 35-40% for short-term gains or ordinary income

#### 2. Loss Threshold Slider (1% - 15%)
- Minimum loss required to trigger harvesting
- Lower thresholds mean more frequent trading
- Higher thresholds may miss opportunities but reduce costs
- Trade-off between:
  * Transaction costs
  * Market tracking error
  * Tax savings potential

## Example Calculation

Let's walk through a concrete example:

Initial Position:
- Security: SPY (S&P 500 ETF)
- Purchase Price: $\$400$
- Current Price: $\$370$
- Quantity: 100 shares
- Holding Period: 60 days
- Tax Rate: 35%
- Loss Threshold: 5%

Calculations:
1. Unrealized Loss % = ($\$370$ - $\$400$) / $\$400$ = -7.5%
2. Dollar Loss = ($\$370$ - $\$400$) × 100 = -$\$3,000$
3. Potential Tax Savings = $\$3,000$ × 0.35 = $\$1,050$

Decision Process:
1. Check Loss Threshold: -7.5% < -5% ✓ (exceeds threshold)
2. Check Wash Sale: 60 days > 30 days ✓ (no wash sale concern)
3. Calculate Tax Benefit: $1,050 in potential tax savings
4. Consider Replacement: Need similar but not identical security (e.g., VOO)

## Practical Considerations

### 1. Trading Costs
- Commission costs (if any)
- Bid-ask spreads
- Market impact of trades
- These would reduce the net benefit of harvesting

### 2. Portfolio Drift
- Selling and buying similar but not identical securities introduces tracking error
- Small differences in performance can compound over time
- Need to balance tax benefits with investment objectives

### 3. Future Tax Rates
- Tax savings today might be more/less valuable than future tax liabilities
- Changes in tax rates affect the value of harvested losses
- Personal circumstances may change

### 4. Market Timing Risk
- Securities sold at a loss might recover quickly
- Replacement securities might not perfectly track original positions
- Wash sale rules limit immediate repurchase options

## Real-World Applications

This simplified example demonstrates core concepts, but actual robo-advisor implementations typically include:

1. More sophisticated wash sale detection
   - Across multiple accounts
   - Consideration of substantially identical securities
   - Integration with dividend reinvestment

2. Advanced optimization
   - Transaction cost modeling
   - Tax lot selection algorithms
   - Cross-security correlations
   - Multi-account household optimization

3. Risk Management
   - Drift limits from target allocation
   - Sector exposure constraints
   - Factor exposure monitoring
   - Liquidity considerations

## Code Implementation Notes

Our Python implementation includes several key classes and functions:

1. `TaxLossHarvester` class
   - Maintains portfolio state
   - Implements harvesting logic
   - Provides interactive visualization

2. Interactive Components
   - Tax rate slider
   - Loss threshold slider
   - Real-time updates
   - Visual feedback

3. Visualization
   - Bar chart for tax savings
   - Scatter plot for loss percentages
   - Combined view for analysis

## Conclusion

Tax loss harvesting is a powerful tool for improving after-tax returns, but requires careful consideration of:
- Trading costs
- Wash sale rules
- Portfolio tracking error
- Individual tax situations

The interactive nature of this example allows exploration of how different parameters affect harvesting opportunities and potential tax savings. This helps understand the sensitivity of tax loss harvesting benefits to various inputs and assumptions.


In [4]:
!jupyter nbextension enable --py widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


In [6]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import ipywidgets as widgets
from IPython.display import display
import plotly.graph_objects as go

In [8]:
class TaxLossHarvester:
    def __init__(self):
        # Create sample portfolio
        self.portfolio = [
            {
                'security': 'SPY',
                'quantity': 100,
                'purchase_price': 400,
                'current_price': 370,
                'purchase_date': datetime.now() - timedelta(days=60)
            },
            {
                'security': 'QQQ',
                'quantity': 50,
                'purchase_price': 380,
                'current_price': 350,
                'purchase_date': datetime.now() - timedelta(days=20)
            },
            {
                'security': 'AAPL',
                'quantity': 200,
                'purchase_price': 150,
                'current_price': 130,
                'purchase_date': datetime.now() - timedelta(days=45)
            }
        ]
        
        # Create interactive widgets
        self.tax_rate_widget = widgets.FloatSlider(
            value=0.35,
            min=0.1,
            max=0.5,
            step=0.01,
            description='Tax Rate:',
            continuous_update=False,
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='50%')
        )

        self.threshold_widget = widgets.FloatSlider(
            value=0.05,
            min=0.01,
            max=0.15,
            step=0.01,
            description='Loss Threshold:',
            continuous_update=False,
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='50%')
        )

        # Create output widget for dynamic updates
        self.output = widgets.Output()

        # Set up the interactive elements
        self.setup_interactive_elements()

    def analyze_portfolio(self, tax_rate, threshold):
        """Analyze portfolio for tax loss harvesting opportunities"""
        opportunities = []
        total_tax_savings = 0

        for position in self.portfolio:
            loss_pct = (position['current_price'] - position['purchase_price']) / position['purchase_price']
            dollar_loss = (position['current_price'] - position['purchase_price']) * position['quantity']
            
            # Only consider positions with losses exceeding threshold
            if abs(loss_pct) > threshold:
                tax_savings = abs(dollar_loss) * tax_rate
                total_tax_savings += tax_savings
                
                opportunities.append({
                    'security': position['security'],
                    'loss_pct': loss_pct,
                    'dollar_loss': dollar_loss,
                    'tax_savings': tax_savings,
                    'days_held': (datetime.now() - position['purchase_date']).days
                })

        return opportunities, total_tax_savings

    def plot_results(self, opportunities):
        """Create visualization of harvesting opportunities"""
        if not opportunities:
            return go.Figure()

        df = pd.DataFrame(opportunities)
        
        fig = go.Figure()
        
        # Bar chart for tax savings
        fig.add_trace(go.Bar(
            x=df['security'],
            y=df['tax_savings'],
            name='Tax Savings ($)',
            marker_color='green'
        ))
        
        # Scatter plot for loss percentages
        fig.add_trace(go.Scatter(
            x=df['security'],
            y=df['loss_pct'] * 100,
            name='Loss (%)',
            yaxis='y2',
            mode='markers',
            marker=dict(size=12, color='red')
        ))
        
        fig.update_layout(
            title='Tax Loss Harvesting Opportunities',
            yaxis=dict(title='Tax Savings ($)', side='left'),
            yaxis2=dict(
                title='Loss (%)',
                side='right',
                overlaying='y',
                tickformat='.1f'
            ),
            showlegend=True,
            height=500
        )
        
        return fig

    def update_analysis(self, change):
        """Update analysis when widgets change"""
        with self.output:
            self.output.clear_output(wait=True)
            
            # Get current widget values
            tax_rate = self.tax_rate_widget.value
            threshold = self.threshold_widget.value
            
            # Analyze portfolio
            opportunities, total_savings = self.analyze_portfolio(tax_rate, threshold)
            
            # Display results
            print(f"\nAnalysis Results (Tax Rate: {tax_rate:.1%}, Threshold: {threshold:.1%})")
            print(f"Total Potential Tax Savings: ${total_savings:,.2f}\n")
            
            if opportunities:
                print("Harvesting Opportunities:")
                for opp in opportunities:
                    print(f"\n{opp['security']}:")
                    print(f"  Loss: {opp['loss_pct']:.1%}")
                    print(f"  Dollar Loss: ${opp['dollar_loss']:,.2f}")
                    print(f"  Tax Savings: ${opp['tax_savings']:,.2f}")
                    print(f"  Days Held: {opp['days_held']}")
                
                # Show plot
                fig = self.plot_results(opportunities)
                fig.show()
            else:
                print("No harvesting opportunities found with current parameters.")

    def setup_interactive_elements(self):
        """Setup the interactive elements of the dashboard"""
        # Observe widget changes
        self.tax_rate_widget.observe(self.update_analysis, names='value')
        self.threshold_widget.observe(self.update_analysis, names='value')
        
        # Show initial view
        display(widgets.VBox([
            widgets.HTML("<h2>Tax Loss Harvesting Analysis</h2>"),
            self.tax_rate_widget,
            self.threshold_widget,
            self.output
        ]))
        
        # Trigger initial update
        self.update_analysis(None)

# Function to create and display the dashboard
def create_tax_loss_harvesting_dashboard():
    return TaxLossHarvester()

In [10]:
# Create and display the dashboard
dashboard = create_tax_loss_harvesting_dashboard()